In [7]:
import os
import tqdm
import wandb
import warnings
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import concurrent.futures

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging
warnings.filterwarnings('ignore', category=UserWarning, module='google.protobuf')

from matplotlib.axes import Axes
from wandb.apis.public import Run

from typing import Union, List, Dict
from src.visualization import set_themes

api = wandb.Api() # Initialize Weights & Biases API, used for fetching run data
set_themes() # Set custom themes for plots
pl.Config.set_tbl_rows(20) # Set Polars table display rows limit

pd.set_option('future.no_silent_downcasting', True)

failed to send, dropping 2 traces to intake at http://localhost:8126/v0.5/traces after 3 retries, 4 additional messages skipped


In [11]:
config = {
    "model": "matrix_factorization",
    "ensure_available_locally": False
}
sorting_criterion = {
    "epoch/test_hitrate@50": 0.5,
    "epoch/test_ndcg@50": 0.25,
}

def fetch_run_metadata(run: Run, considered_metrics: Union[str, Dict[str, float]] = "epoch/epoch") -> Dict:
    run_config = {}
    for key, value in run.config.items():
        # Convert lists and dicts to strings
        if isinstance(value, (list, dict)):
            run_config[key] = str(value)
        else:
            run_config[key] = value

    run_history = run.history()
    run_history = run_history.replace({"Infinity": np.inf, "NaN": np.nan})

    if isinstance(considered_metrics, str):
        run_history["score"] = run_history[considered_metrics]
    elif isinstance(considered_metrics, dict):
        run_history["score"] = sum(
            run_history[metric] * weight for metric, weight in considered_metrics.items()
        )
    else:
        raise ValueError("considered_metrics must be either a string or a dictionary")
    
    best_summary = run_history.iloc[run_history["score"].argmax()]
    best_summary = {f"best:{key}": val for key, val in best_summary.items()}
    
    return {
        "run_id": run.id,
        "run_name": run.name,
        "sweep_id": run.sweep.id if run.sweep else None,
        "model": run.config.get("model"),
        **run_config,
        **{metric: run_history[metric].to_list() for metric in run_history},
        **best_summary,
    }

batch_size = 8
records = []
futures = {}
executor = concurrent.futures.ThreadPoolExecutor(max_workers=batch_size)
runs:List[Run] = api.runs("feedr/peppermint-matrix", per_page=16, filters={"config.model": config["model"]})
run_iterator = iter(runs)
with tqdm.tqdm(total=len(runs), ncols=128) as pbar:
    while len(records) < len(runs):
        # submit new tasks if we empty slots in the batch
        while len(futures) < batch_size and len(records) + len(futures) < len(runs):
            current_runs = next(run_iterator)
            current_future = executor.submit(fetch_run_metadata, current_runs, sorting_criterion)
            futures[current_future] = current_runs

        # check for completed tasks
        finished_futures, _ = concurrent.futures.wait(futures.keys(), return_when=concurrent.futures.FIRST_COMPLETED, timeout=0.1)
        for finished_future in finished_futures:
            finished_run = futures.pop(finished_future)
            records.append(finished_future.result())
            pbar.update(1)

# Create a Polars DataFrame from the records
experiment_runs = pl.DataFrame(records, infer_schema_length=None)
    
# # Tag run as available locally if the model files exist
local_run_ids = []
local_sweep_ids = os.listdir(f"./models/{config['model']}/")
for sweep_id in local_sweep_ids:
    local_run_ids.extend([run_id for run_id in os.listdir(f"./models/{config['model']}/{sweep_id}/")])
    
experiment_runs = experiment_runs.with_columns(
    available_locally=pl.col("run_id").is_in(local_run_ids)
)

if config["ensure_available_locally"]:
    experiment_runs = experiment_runs.filter(pl.col("available_locally") == True)

experiment_runs = experiment_runs.sort("_timestamp", descending=False)
experiment_runs.select(
    pl.col("run_id"),
    pl.col("run_name"),
    pl.col("sweep_id"),
    pl.col("model"),
    pl.col("embedding_dimension"),
    pl.col("shuffle"),
    pl.col("best:epoch/epoch"),
    pl.col("best:epoch/train_loss"),
    pl.col("best:epoch/test_loss"),
    pl.col("best:epoch/test_recall@10"),
    pl.col("best:epoch/test_ndcg@10"),
)

  0%|                                                                                                    | 0/76 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 76/76 [00:05<00:00, 13.01it/s]


run_id,run_name,sweep_id,model,embedding_dimension,shuffle,best:epoch/epoch,best:epoch/train_loss,best:epoch/test_loss,best:epoch/test_recall@10,best:epoch/test_ndcg@10
str,str,str,str,i64,bool,f64,f64,f64,f64,f64
"""sm4faeql""","""electric-sweep-1""","""pjxpg4bb""","""matrix_factorization""",16,false,51.0,0.044853,0.223118,0.022528,0.087484
"""mhh2vww3""","""solar-sweep-3""","""pjxpg4bb""","""matrix_factorization""",4,true,46.0,0.092834,0.20148,0.019505,0.076231
"""im0wjtep""","""lively-sweep-2""","""pjxpg4bb""","""matrix_factorization""",32,true,26.0,0.039394,0.196034,0.022696,0.088301
"""yszchdsw""","""laced-sweep-4""","""pjxpg4bb""","""matrix_factorization""",32,true,33.0,0.035967,0.219474,0.022535,0.086142
"""gkt242cc""","""clean-sweep-5""","""pjxpg4bb""","""matrix_factorization""",256,true,11.0,0.027117,0.208644,0.020824,0.082289
"""yvji2dcx""","""rose-sweep-6""","""pjxpg4bb""","""matrix_factorization""",1024,false,1.0,0.2362,0.242782,0.023828,0.091777
"""n9r3cmud""","""warm-sweep-7""","""pjxpg4bb""","""matrix_factorization""",128,false,7.0,0.040676,0.183099,0.022164,0.085856
"""eg5qej8g""","""twilight-sweep-8""","""pjxpg4bb""","""matrix_factorization""",256,true,14.0,0.022493,0.22761,0.02086,0.082151
"""tw2l0zlb""","""fallen-sweep-9""","""pjxpg4bb""","""matrix_factorization""",1024,true,0.0,0.54582,0.310808,0.021982,0.084718


# TL;DR

In [27]:
experiment_summary = experiment_runs.group_by("embedding_dimension", "shuffle").agg(
    pl.col("best:epoch/epoch").mean(),
    pl.col("best:epoch/test_recall@10").mean(),
    pl.col("best:epoch/test_ndcg@10").mean()
).sort("embedding_dimension", "shuffle")
experiment_summary

embedding_dimension,shuffle,best:epoch/epoch,best:epoch/test_recall@10,best:epoch/test_ndcg@10
i64,bool,f64,f64,f64
2,false,51.5,0.014958,0.058565
2,true,60.0,0.01476,0.057768
4,true,51.0,0.019432,0.075738
8,false,44.333333,0.021928,0.084803
8,true,56.5,0.022316,0.085669
16,false,47.666667,0.022441,0.087474
16,true,34.0,0.022575,0.087367
32,false,16.833333,0.023091,0.089375
32,true,27.333333,0.022707,0.087375


In [28]:
experiment_summary[["embedding_dimension", "shuffle", "best:epoch/epoch"]].pivot(
    values=["best:epoch/epoch"],
    index="embedding_dimension",
    columns="shuffle"
)

/tmp/ipykernel_84096/1569947604.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "shuffle", "best:epoch/epoch"]].pivot(


embedding_dimension,false,true
i64,f64,f64
2,51.5,60.0
4,null,51.0
8,44.333333,56.5
16,47.666667,34.0
32,16.833333,27.333333
64,12.5,19.75
128,8.333333,16.333333
256,9.0,11.5
512,1.0,11.5


In [30]:
experiment_summary[["embedding_dimension", "shuffle", "best:epoch/test_recall@10"]].pivot(
    values=["best:epoch/test_recall@10"],
    index="embedding_dimension",
    columns="shuffle"
).with_columns(
    does_improve = pl.col("true") > pl.col("false")
)

/tmp/ipykernel_84096/1599131782.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "shuffle", "best:epoch/test_recall@10"]].pivot(


embedding_dimension,false,true,does_improve
i64,f64,f64,bool
2,0.014958,0.01476,false
4,null,0.019432,null
8,0.021928,0.022316,true
16,0.022441,0.022575,true
32,0.023091,0.022707,false
64,0.022313,0.022219,false
128,0.022334,0.021442,false
256,0.021256,0.020958,false
512,0.024152,0.019999,false


In [31]:
experiment_summary[["embedding_dimension", "shuffle", "best:epoch/test_ndcg@10"]].pivot(
    values=["best:epoch/test_ndcg@10"],
    index="embedding_dimension",
    columns="shuffle"
).with_columns(
    does_improve = pl.col("true") > pl.col("false")
)

/tmp/ipykernel_84096/4150577613.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  experiment_summary[["embedding_dimension", "shuffle", "best:epoch/test_ndcg@10"]].pivot(


embedding_dimension,false,true,does_improve
i64,f64,f64,bool
2,0.058565,0.057768,false
4,null,0.075738,null
8,0.084803,0.085669,true
16,0.087474,0.087367,false
32,0.089375,0.087375,false
64,0.08677,0.086439,false
128,0.086431,0.083873,false
256,0.083759,0.082423,false
512,0.091344,0.078317,false
